#  Food truck business in San Antonio TX


## Table of contents
* [Introduction: Starting a food truck snack business](#introduction)
* [Problem: Where to park my food truck in San Antonio, TX? ](#problem)
* [Data: The usage of date to find the best place](#data)
* [Methodology: Clustering venues to detect the highest concentration area](#clustering)
* [Results and Discussion: Map creation and final](#map)
* [Conclusion](#conclusion)

##### Introduction: Starting a food truck snack business <a name="introduction"></a>

Starting a business besides your current job is always a good idea, now a days the salaries are not always enough to afford the living of a family or just trying to improve our income so we have the chance to access a better life but also at the beginning we do not want to quit our job because it represents a consistent source of money. In my personal opinion Food trucks are a good option because help us to start our business avoiding the expensive leasing in most of the cities, it also gives flexibility to change location depending on the day of the week, the season, work in fairs, just because of looking for a place with more possible costumers or bad weather in some places.

##### Problem: Where to park my food truck in San Antonio, TX? <a name="problem"></a>

All this idea comes from the fact that I am recently married to my  wife who is from San Antonio TX so we plan to move there then I am using this city as location for my project. This idea is mainly about a fast way to provide snacks to people who are basically walking in any place or doing something that takes time for them to be hungry but at the same time they do not want to go to a restaurant so they can get a quick snack!

The must complicated question is where to park our food truck?

##### Data: The usage of date to find the best place <a name="data"></a>

The idea is to find places that have the same market and or places that may have high amount of people who is doing certain activity and that meanwhile may get hungry but they have no time to sit in a restaurant. 

Places will be:

Food trucks 

Snacks

Desserts

Markets


Places in all of these categories will be searched, because not all of them represent the same amount of chance for adding costumers they will be given with different score, the one on top representing the highest.

Clustering by the distance and then the cluster that according to its score represents the highest possibility for the the food truck to succeed will be chosen. 

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge --no-deps altair --yes

!conda install -c conda-forge --no-deps vincent --yes

!conda install -c conda-forge geopy=1.19.0 --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge --no-deps folium=0.5.0 --yes
import folium # plotting library

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



##### Example of dataset needed

In [2]:
CLIENT_ID = 'PAB4VUBIYDALJMJHJMP3QQ0BX32KT1TMUBPKL3IMCSQYI0WX' # your Foursquare ID
CLIENT_SECRET = 'T04KIP2AECI2BDDMB2QO2KW4QCL5AANWWYZX1ORDGOC2D1YC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PAB4VUBIYDALJMJHJMP3QQ0BX32KT1TMUBPKL3IMCSQYI0WX
CLIENT_SECRET:T04KIP2AECI2BDDMB2QO2KW4QCL5AANWWYZX1ORDGOC2D1YC


In [3]:
address = 'San Antonio, TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

29.4246002 -98.4951405


In [4]:
search_query = 'Food truck'
radius = 20000

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PAB4VUBIYDALJMJHJMP3QQ0BX32KT1TMUBPKL3IMCSQYI0WX&client_secret=T04KIP2AECI2BDDMB2QO2KW4QCL5AANWWYZX1ORDGOC2D1YC&ll=29.4246002,-98.4951405&v=20180604&query=Food truck&radius=20000&limit=30'

In [7]:
results_FT = requests.get(url).json()
results_FT

{'meta': {'code': 200, 'requestId': '5e8d25729388d70028b2de93'},
 'response': {'venues': [{'id': '504799e5e4b0aa9a70b2a79a',
    'name': "Cheesy Jane's Food Truck",
    'location': {'lat': 29.424348952586705,
     'lng': -98.49393914683124,
     'labeledLatLngs': [{'label': 'display',
       'lat': 29.424348952586705,
       'lng': -98.49393914683124}],
     'distance': 119,
     'postalCode': '78205',
     'cc': 'US',
     'city': 'San Antonio',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['San Antonio, TX 78205', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1cb941735',
      'name': 'Food Truck',
      'pluralName': 'Food Trucks',
      'shortName': 'Food Truck',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586308586',
    'hasPerk': False},
   {'id': '4fd8dbafe4b06071ea86e452',
    'name': 'Blazin Burgers Food Truck',
    '

In [8]:
# assign relevant part of JSON to venues
venues_FT = results_FT['response']['venues']

# tranform venues into a dataframe
FT = json_normalize(venues_FT)
FT.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet
0,504799e5e4b0aa9a70b2a79a,Cheesy Jane's Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1586308586,False,29.424349,-98.493939,"[{'label': 'display', 'lat': 29.42434895258670...",119,78205,US,San Antonio,TX,United States,"[San Antonio, TX 78205, United States]",NaN,NaN
1,4fd8dbafe4b06071ea86e452,Blazin Burgers Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1586308586,False,29.423200,-98.494582,"[{'label': 'display', 'lat': 29.42319965982001...",165,78205,US,San Antonio,TX,United States,"[San Antonio, TX 78205, United States]",NaN,NaN
2,5041523ce4b0f7ca2bf5cebe,Rackspace food truck friday,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1586308586,False,29.425911,-98.493709,"[{'label': 'display', 'lat': 29.42591063303164...",201,78212,US,San Antonio,TX,United States,"[San Antonio, TX 78212, United States]",NaN,NaN
3,58e85f4d54073d7d2da0284d,Ni Entiendo Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1586308586,False,29.417665,-98.488593,"[{'label': 'display', 'lat': 29.41766500000000...",999,78205,US,San Antonio,TX,United States,"[609 S Alamo St, San Antonio, TX 78205, United...",609 S Alamo St,NaN
4,5afdbb03f8cbd40039fcab35,Eddie Hijo-E-Su Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1586308586,False,29.432573,-98.499962,"[{'label': 'display', 'lat': 29.432573, 'lng':...",1003,78212,US,San Antonio,TX,United States,"[Camaron @ Elmira, San Antonio, TX 78212, Unit...",NaN,Camaron @ Elmira


This an example of the places that may help us to find the kind of places that we are interested on, with the latitude and longitude of the places the idea is the create a map where we can see all of them at the same time and decide in which area we can find most of them.

In [9]:
FT.drop(['id', 'categories', 'referralId', 'location.labeledLatLngs', 'hasPerk', 'location.postalCode', 'location.formattedAddress', 
         'location.address', 'location.crossStreet', 'location.cc', 'location.distance'], axis=1, inplace=True)

In [10]:
#FT.drop(['Kind'], axis=1, inplace= True)
#FT.insert(6, "Kind of place", True) 
FT['Kind of place']='Food Truck'
FT.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)

In [11]:
FT

,name,lat,lng,location.city,location.state,location.country,Kind of place
0,Cheesy Jane's Food Truck,29.424349,-98.493939,San Antonio,TX,United States,Food Truck
1,Blazin Burgers Food Truck,29.423200,-98.494582,San Antonio,TX,United States,Food Truck
2,Rackspace food truck friday,29.425911,-98.493709,San Antonio,TX,United States,Food Truck
3,Ni Entiendo Food Truck,29.417665,-98.488593,San Antonio,TX,United States,Food Truck
4,Eddie Hijo-E-Su Food Truck,29.432573,-98.499962,San Antonio,TX,United States,Food Truck
5,Food Trucks @ Weston Wedneday,29.429246,-98.491511,San Antonio,TX,United States,Food Truck
6,Food Truck,29.478451,-98.587252,San Antonio,TX,United States,Food Truck
7,totally shredded food truck,29.426317,-98.541107,San Antonio,TX,United States,Food Truck
8,Chatmans Food Truck,29.449317,-98.455673,San Antonio,TX,United States,Food Truck
9,Fort Sam Food Truck,29.454201,-98.453625,Fort Sam Houston,TX,United States,Food Truck


This is the example of a table of clean data that we can work with, in this example we only got Food truck venues, in next cell we will do the same for the other venues.

In [15]:
#Get data for venues under 'snack'
search_query = 'Snack'
radius = 20000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results_SNACK = requests.get(url).json()
# assign relevant part of JSON to venues
venues_SNACK = results_SNACK['response']['venues']
# tranform venues into a dataframe
SNACK = json_normalize(venues_SNACK)
SNACK.drop(['id', 'categories', 'referralId', 'location.labeledLatLngs', 'hasPerk', 'location.postalCode', 'location.formattedAddress', 
         'location.address', 'location.crossStreet', 'location.cc', 'location.distance'
            ,'location.neighborhood'], axis=1, inplace=True)

SNACK.insert(6, "Kind of place", True) 
SNACK['Kind of place']='Snack'
SNACK.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)

#Get data for venues under 'desserts'
search_query = 'desserts'
radius = 20000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results_DESSERT = requests.get(url).json()
# assign relevant part of JSON to venues
venues_DESSERT = results_DESSERT['response']['venues']
# tranform venues into a dataframe
DESSERT = json_normalize(venues_DESSERT)
DESSERT.drop(['id', 'categories', 'referralId', 'location.labeledLatLngs', 'hasPerk', 'location.postalCode', 'location.formattedAddress', 
         'location.address', 'location.crossStreet', 'location.cc', 'location.distance','venuePage.id'], axis=1, inplace=True)

DESSERT.insert(6, "Kind of place", True) 
DESSERT['Kind of place']='Dessert'
DESSERT.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)

#Get data for venues under 'market'
search_query = 'market'
radius = 20000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results_MARKET = requests.get(url).json()
# assign relevant part of JSON to venues
venues_MARKET = results_MARKET['response']['venues']
# tranform venues into a dataframe
MARKET = json_normalize(venues_MARKET)
MARKET.drop(['id', 'categories', 'referralId', 'location.labeledLatLngs', 'hasPerk', 'location.postalCode', 'location.formattedAddress', 
        'location.address', 'location.crossStreet', 'location.cc', 'location.distance','venuePage.id', 
        'location.neighborhood', 'delivery.id', 'delivery.provider.icon.name', 'delivery.provider.icon.sizes',
        'delivery.url', 'delivery.provider.name', 'delivery.provider.icon.prefix'], axis=1, inplace=True)

MARKET.insert(6, "Kind of place", True) 
MARKET['Kind of place']='Market'
MARKET.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


##### Methodology: Clustering venues to detect the highest concentration area <a name="clustering"></a>

All the data collected needs to be gathered into a single Total data frame and then create a data frame that only the location of every place with the one the clustering will be done by using kmeans.

In [17]:
Total=FT.append(SNACK) 
Total=Total.append(DESSERT)
Total=Total.append(MARKET)

Location_only=Total.drop(['Kind of place', 'name', 'location.city',
                             'location.state', 'location.country'], 1)

# set number of clusters
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Location_only)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 1, 1, 0, 4, 1, 1], dtype=int32)

This column will be added to the Total data frame so all info can finally be gathered.

In [18]:
Total.insert(0, 'Cluster Labels', kmeans.labels_)
Total

,Cluster Labels,name,lat,lng,location.city,location.state,location.country,Kind of place
0,1,Cheesy Jane's Food Truck,29.424349,-98.493939,San Antonio,TX,United States,Food Truck
1,1,Blazin Burgers Food Truck,29.423200,-98.494582,San Antonio,TX,United States,Food Truck
2,1,Rackspace food truck friday,29.425911,-98.493709,San Antonio,TX,United States,Food Truck
3,1,Ni Entiendo Food Truck,29.417665,-98.488593,San Antonio,TX,United States,Food Truck
4,1,Eddie Hijo-E-Su Food Truck,29.432573,-98.499962,San Antonio,TX,United States,Food Truck
...,...,...,...,...,...,...,...,...
25,3,Ali Baba International Food Market,29.529804,-98.564797,San Antonio,TX,United States,Market
26,1,Historic Market Square,29.425261,-98.499744,San Antonio,TX,United States,Market
27,1,Market Square Visitor Center,29.425314,-98.485214,San Antonio,TX,United States,Market
28,1,Chas Market & Kitchen,29.437578,-98.470522,San Antonio,TX,United States,Market


##### Results and Discussion: Map creation and final   <a name="map"></a>

In [69]:

SA_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around SA

# add a red circle marker to represent SA
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='San Antonio',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(SA_map)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Total['lat'], Total['lng'], Total['Kind of place'], Total['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(SA_map)
       
SA_map

In [68]:
Total.groupby(['Cluster Labels']).count()

,name,lat,lng,location.city,location.state,location.country,Kind of place
Cluster Labels,,,,,,,
0,11,11,11,10,11,11,11
1,40,40,40,38,40,40,40
2,17,17,17,14,17,17,17
3,17,17,17,15,17,17,17
4,12,12,12,12,12,12,12


##### Conclusion <a name="conclusion"></a>

After all was not quite necessary to establish score for every kind of place because at the end the difference of concentration on every cluster was huge, there is no doubt that cluster 0 or Down Town are has a lot of interesting places to located our food truck.

Also is necessary to include that a further analysis of this places would be needed because most of them are not completely public or allow any kind of food business within their area, also information related to fees can change our decision.